# Imports

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Collecting data

In [27]:
df_covid = pd.read_html("https://www.statista.com/statistics/1104709/coronavirus-deaths-worldwide-per-million-inhabitants/")[0]

In [28]:
df_covid.head(1)

,Unnamed: 0,Confirmed cases (absolute),Cases in last 7 days,Confirmed deaths (absolute),Deaths in last 7 days,Daily increase (# deaths),Population (in millions),Deaths per million (total),Deaths per million (last 7 days)
0,Belgium,757696,13814,21956,163,33,11.48,1911.87,14.19


In [29]:
df_covid.rename(columns={"Unnamed: 0":"Country"}, inplace=True)

In [30]:
df_covid.head()

,Country,Confirmed cases (absolute),Cases in last 7 days,Confirmed deaths (absolute),Deaths in last 7 days,Daily increase (# deaths),Population (in millions),Deaths per million (total),Deaths per million (last 7 days)
0,Belgium,757696,13814,21956,163,33,11.48,1911.87,14.19
1,Czechia,1168491,56169,19537,941,207,10.67,1831.07,88.19
2,Slovenia,185916,4310,3792,47,8,2.09,1816.14,22.51
3,United Kingdom¹,4122518,63244,121074,2367,548,66.83,1811.55,35.42
4,Italy,2832162,80505,96348,1808,356,60.30,1597.88,29.98


# Analyze

In [23]:
df_covid.sort_values("Deaths per million (last 7 days)", ascending=False).head()

,Unnamed: 0,Confirmed cases (absolute),Cases in last 7 days,Confirmed deaths (absolute),Deaths in last 7 days,Daily increase (# deaths),Population (in millions),Deaths per million (total),Deaths per million (last 7 days)
19,Slovakia,294790,11926,6671,503,94,5.45,1223.12,92.22
1,Czechia,1168491,56169,19537,941,207,10.67,1831.07,88.19
9,Hungary,407274,16104,14450,519,103,9.77,1479.03,53.12
11,Moldova,178540,5725,3823,126,3823,2.66,1438.50,47.41
41,Lebanon,359337,13257,4446,294,59,6.86,648.51,42.88


In [31]:
df_covid[["Cases in last 7 days", "Deaths in last 7 days"]].corr()

,Cases in last 7 days,Deaths in last 7 days
Cases in last 7 days,1.000000,0.944094
Deaths in last 7 days,0.944094,1.000000


In [35]:
df_covid[["Deaths in last 7 days", "Population (in millions)"]].corr()

,Deaths in last 7 days,Population (in millions)
Deaths in last 7 days,1.000000,0.198672
Population (in millions),0.198672,1.000000


# Prediction

In [39]:
from sklearn import linear_model

# Split the data into training/testing sets
df_covid_train = df_covid[["Cases in last 7 days", "Deaths in last 7 days"]][:100]
df_covid_test = df_covid[["Cases in last 7 days", "Deaths in last 7 days"]][100:]

# Split the data into training/testing sets
df_covid_X_train = df_covid_train[["Cases in last 7 days"]]
df_covid_X_test = df_covid_test[["Cases in last 7 days"]]

# Split the targets into training/testing sets
df_covid_y_train = df_covid_train[["Deaths in last 7 days"]]
df_covid_y_test = df_covid_test[["Deaths in last 7 days"]]

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(df_covid_X_train, df_covid_y_train)

# Make predictions using the testing set
df_covid_y_pred = regr.predict(df_covid_X_test)

In [41]:
df_covid_test['prediction'] = df_covid_y_pred

In [42]:
df_covid_test.head()

,Cases in last 7 days,Deaths in last 7 days,prediction
100,2239,60,50.112264
101,1471,63,30.314749
102,2667,28,61.145254
103,127,12,-4.330901
104,27,0,-6.908702


In [48]:
df_covid_test['prediction'] = df_covid_test['prediction'].apply(lambda x: 0 if x<0 else x)

In [49]:
df_covid_test.head()

,Cases in last 7 days,Deaths in last 7 days,prediction
100,2239,60,50.112264
101,1471,63,30.314749
102,2667,28,61.145254
103,127,12,0.000000
104,27,0,0.000000


In [50]:
df_covid_test.diff = abs(df_covid_test['Deaths in last 7 days'] - df_covid_test['prediction'])

In [51]:
df_covid_test.diff.describe()

count     52.000000
mean      22.539932
std       62.656929
min        0.000000
25%        0.000000
50%        3.407873
75%       12.373997
max      335.544855
dtype: float64